<a href="https://colab.research.google.com/github/jeetu-g2-1/Stock-invest-ai-agent/blob/main/stockinvestrecommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collect user data

In [ ]:
user_profile = {
    'risk_tolerance': 'moderate',
    'investment_goals': 'long_term_growth',
    'total_investment': 10000
}


# Fetch and analyze market data

In [ ]:
import requests

def get_company_data(symbol):
    api_key = '1HRCG3M2TLBM8PIS'
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    return response.json()

company_list = ['AAPL', 'MSFT', 'GOOGL']
company_data = [get_company_data(symbol) for symbol in company_list]


# Recommendations

In [ ]:
def clean_value(value):
    try:
        return float(value)
    except (TypeError, ValueError):
        return 0.0

def heuristic_score(company):
    market_cap = clean_value(company.get('MarketCapitalization'))
    pe_ratio = clean_value(company.get('PERatio'))
    dividend_yield = clean_value(company.get('DividendYield'))
    earnings_growth = clean_value(company.get('EPS')) / max(clean_value(company.get('PreviousClose')), 1)

    # Simple heuristic: higher is better for dividend_yield and earnings_growth, lower is better for pe_ratio
    score = (dividend_yield + earnings_growth) / (pe_ratio + 1)  # Avoid division by zero
    return score

def recommend_investment(amount, companies_data):
    investment_recommendations = {}
    for company in companies_data:
        score = heuristic_score(company)
        investment_recommendations[company['Symbol']] = score

    total_score = sum(investment_recommendations.values())
    for company in investment_recommendations:
        investment_recommendations[company] = (investment_recommendations[company] / total_score) * amount

    return investment_recommendations

# Example usage
recommendations = recommend_investment(user_profile['total_investment'], company_data)
for company, amount in recommendations.items():
    print(f"Invest ${amount:.2f} in {company}")


Invest $2815.30 in AAPL
Invest $4036.38 in MSFT
Invest $3148.32 in GOOGL


# Output of recommendations

In [ ]:
for company, amount in recommendations.items():
    print(f"Invest ${amount:.2f} in {company}")


Invest $2815.30 in AAPL
Invest $4036.38 in MSFT
Invest $3148.32 in GOOGL


# Code explanation
clean_value Function: This function attempts to convert a value to a float. If it encounters an error (e.g., the value is 'None' or not a number), it returns 0.0. This ensures that all numerical operations can proceed without throwing exceptions.
heuristic_score Function: Uses clean_value to safely convert company data fields to floats, ensuring calculations proceed without errors.
recommend_investment Function: Iterates over the company data, calculates a heuristic score for each company, and then distributes the investment amount proportionally based on these scores.
